In [1]:
import pandas as pd
import numpy as np
import multiprocessing

from fbprophet import Prophet
from datetime import datetime
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_error as mae
from joblib import Parallel, delayed
from tqdm import tqdm

import numpy as np
import scipy.stats

In [20]:
def preprocess_download(data, start_date_back, end_date_back):
    data = data.dropna(subset=['subregion1_code'])
    data = data[data.country_code == 'US']

    data = data[['date',
                 'subregion1_code',
                 'total_deceased',
                 'average_temperature', 
                 'minimum_temperature',
                 'maximum_temperature', 
                 'rainfall', 
                 'dew_point', 
                 'relative_humidity',
                 'new_confirmed']]
    
    data['days_back'] = [(datetime.now() - day).days for day in pd.to_datetime(data.date)]
    data = data[data['days_back']<=start_date_back]
    data = data[data['days_back']>=end_date_back]
    data.fillna(0, inplace = True)
    
    res = {}
    for state in data.subregion1_code.unique():
        if state in ['MP', 
                     'AS',
                     'GU',
                     'PR', 
                     'VI']:
            continue
        print('State: ', state)
        tmp_state = data[data.subregion1_code == state]
        tmp = []
        for date in tmp_state.date.unique():
            tmp_state_day = tmp_state[tmp_state.date == date]
            tmp.append({'ds': pd.to_datetime(date), 
                        'total_deceased': sum(tmp_state_day.total_deceased),
                        'average_temperature': np.mean(tmp_state_day.average_temperature),
                        'minimum_temperature': np.mean(tmp_state_day.minimum_temperature),
                        'maximum_temperature': np.mean(tmp_state_day.maximum_temperature),
                        'rainfall': np.mean(tmp_state_day.rainfall),
                        'dew_point': np.mean(tmp_state_day.dew_point),
                        'relative_humidity': np.mean(tmp_state_day.relative_humidity),
                        'y': sum(tmp_state_day.new_confirmed)})
        res[state] = pd.DataFrame(tmp)
    
    return(res)

In [21]:
data = preprocess_download(pd.read_csv("https://storage.googleapis.com/covid19-open-data/v2/main.csv", 
                                       low_memory=False), 
                           62, 
                           2)


State:  AK
State:  AL
State:  AR
State:  AS
State:  AZ
State:  CA
State:  CO
State:  CT
State:  DC
State:  DE
State:  FL
State:  GA
State:  GU
State:  HI
State:  IA
State:  ID
State:  IL
State:  IN
State:  KS
State:  KY
State:  LA
State:  MA
State:  MD
State:  ME
State:  MI
State:  MN
State:  MO
State:  MP
State:  MS
State:  MT
State:  NC
State:  ND
State:  NE
State:  NH
State:  NJ
State:  NM
State:  NV
State:  NY
State:  OH
State:  OK
State:  OR
State:  PA
State:  PR
State:  RI
State:  SC
State:  SD
State:  TN
State:  TX
State:  UT
State:  VA
State:  VI
State:  VT
State:  WA
State:  WI
State:  WV
State:  WY


In [55]:
def run_prophet(data, forecast_dist, state):
    def mean_confidence_interval(data, confidence=0.95):
        a = 1.0 * np.array(data)
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return h
    
    def Prophet_reg(df):
        prophet = Prophet(yearly_seasonality=False, daily_seasonality=True)
        for col in df.columns:
            if col not in ['ds','y']:
                prophet.add_regressor(col)
        prophet.fit(df)
        return prophet
    
    def make_future_dataframe(df):
        future = prophet.make_future_dataframe(periods=forecast_dist, freq='D')

        for col in df.columns:
            if col not in ['ds','y']:
                future[col] = df[col]
        future.fillna(0, inplace = True)
        return future
    
    df = data[state]
    
    y_train_1, y_test_1 = temporal_train_test_split(df, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_1)
    future = make_future_dataframe(y_train_1)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_1):]
    oot_val_1 = (y_test_1['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_2, y_test_2 = temporal_train_test_split(y_train_1, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_2)
    future = make_future_dataframe(y_train_2)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_2):]
    oot_val_2 = (y_test_2['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_3, y_test_3 = temporal_train_test_split(y_train_2, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_3)
    future = make_future_dataframe(y_train_3)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_3):]
    oot_val_3 = (y_test_3['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_4, y_test_4 = temporal_train_test_split(y_train_3, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_4)
    future = make_future_dataframe(y_train_4)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_4):]
    oot_val_4 = (y_test_4['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_5, y_test_5 = temporal_train_test_split(y_train_4, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_5)
    future = make_future_dataframe(y_train_5)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_5):]
    oot_val_5 = (y_test_5['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    ci = mean_confidence_interval([oot_val_1, oot_val_2, oot_val_3, oot_val_4, oot_val_5])
    
    prophet = Prophet_reg(df)
    future = make_future_dataframe(df)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(df):]
    pred_future = fb_preds.values[-1:][0]
    return {'State': state,
            'ci_Prediction_%': np.round(ci/pred_future*100),
            'Prediction_95_ci_l': pred_future-ci,
            'Prediction': pred_future,  
            'Prediction_95_ci_h': pred_future+ci}

In [56]:
num_cores = multiprocessing.cpu_count()
state_list = tqdm(list([state for state in data.keys()]))
processed_list = Parallel(n_jobs=num_cores)(delayed(run_prophet)(data, 
                                                                 3, 
                                                                 state) for state in state_list)

100%|██████████| 56/56 [00:30<00:00,  1.81it/s]


In [57]:
state_level_predictions = pd.DataFrame(processed_list)
# state_level_predictions.sort_values(['Prediction'], ascending = False)
# state_level_predictions.sort_values(['ci_Prediction_%'], ascending = False)
state_level_predictions.sort_values(['Prediction'], ascending = True)

,State,ci_Prediction_%,Prediction_95_ci_l,Prediction,Prediction_95_ci_h
16,IL,-18.0,-245236.973975,-208649.171529,-172061.369084
21,MA,-38.0,-259862.442639,-188090.701933,-116318.961226
24,MI,-46.0,-231787.610710,-158779.025993,-85770.441275
7,CT,-39.0,-120962.215273,-86911.917343,-52861.619414
41,PA,-11.0,-62453.667051,-56321.695012,-50189.722974
17,IN,-58.0,-68957.856205,-43656.183659,-18354.511113
46,TN,-48.0,-46927.997942,-31612.142171,-16296.286401
25,MN,-38.0,-30676.339037,-22192.734623,-13709.130208
44,SC,-44.0,-30122.101781,-20878.759047,-11635.416313
43,RI,-47.0,-29486.124130,-20080.248869,-10674.373607


In [19]:
state_level_predictions.to_csv('./statepreds.csv')